In [65]:
with open('input.txt','r') as f:
    text = f.read()





In [2]:
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [77]:
# 词典、编码器(函数)、解码器（函数）
chars = sorted(list(set(text)))
vacob_size = len(chars)
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [67]:
# 文本转换为索引
import torch

data = torch.tensor(encode(text), dtype=torch.long)


In [68]:
# 拆分数据集
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

In [69]:
# 训练文本采样长度（演示）
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
print(x)
print(y)
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'输入内容：{context}, 预测的目标：{target}')


tensor([18, 47, 56, 57, 58,  1, 15, 47])
tensor([47, 56, 57, 58,  1, 15, 47, 58])
输入内容：tensor([18]), 预测的目标：47
输入内容：tensor([18, 47]), 预测的目标：56
输入内容：tensor([18, 47, 56]), 预测的目标：57
输入内容：tensor([18, 47, 56, 57]), 预测的目标：58
输入内容：tensor([18, 47, 56, 57, 58]), 预测的目标：1
输入内容：tensor([18, 47, 56, 57, 58,  1]), 预测的目标：15
输入内容：tensor([18, 47, 56, 57, 58,  1, 15]), 预测的目标：47
输入内容：tensor([18, 47, 56, 57, 58,  1, 15, 47]), 预测的目标：58


In [70]:
# torch.manual_seed(1337)  #随机数种子，仅实验复现实验场景和结果使用
# 模型的训练数据集
block_size = 8
batch_size = 4

# 动态从数据集中选择位置索引
# ix

def get_batch(split):
    # 选择数据集或者验证集
    data = train_data if split == 'train' else val_data

    ix = torch.randint(len(train_data) - block_size, (batch_size, ) ) #【0， 103846】随机生成位置索引，向后截取block_size字符巡检
    x = torch.stack([train_data[i:i+block_size] for i in ix])
    y = torch.stack([train_data[i+1:i+block_size+1] for i in ix])
    return x, y


In [71]:
get_batch('train')

(tensor([[58, 46, 47, 57,  1, 58, 56, 53],
         [53, 56,  1, 58, 46, 39, 58,  1],
         [ 1, 58, 46, 53, 59,  6,  1, 49],
         [52, 45, 43, 50, 53,  1, 47, 57]]),
 tensor([[46, 47, 57,  1, 58, 56, 53, 59],
         [56,  1, 58, 46, 39, 58,  1, 21],
         [58, 46, 53, 59,  6,  1, 49, 43],
         [45, 43, 50, 53,  1, 47, 57,  1]]))

In [72]:
x, y = get_batch('train')
x.shape
y.shape

for b in range(batch_size):
    for t in range(block_size):
        context = x[b, :t+1]
        target = y[b, t]
        print(f'输入内容：{context}, 预测的目标：{target}')

输入内容：tensor([39]), 预测的目标：60
输入内容：tensor([39, 60]), 预测的目标：43
输入内容：tensor([39, 60, 43]), 预测的目标：1
输入内容：tensor([39, 60, 43,  1]), 预测的目标：51
输入内容：tensor([39, 60, 43,  1, 51]), 预测的目标：43
输入内容：tensor([39, 60, 43,  1, 51, 43]), 预测的目标：0
输入内容：tensor([39, 60, 43,  1, 51, 43,  0]), 预测的目标：39
输入内容：tensor([39, 60, 43,  1, 51, 43,  0, 39]), 预测的目标：61
输入内容：tensor([15]), 预测的目标：21
输入内容：tensor([15, 21]), 预测的目标：27
输入内容：tensor([15, 21, 27]), 预测的目标：10
输入内容：tensor([15, 21, 27, 10]), 预测的目标：0
输入内容：tensor([15, 21, 27, 10,  0]), 预测的目标：13
输入内容：tensor([15, 21, 27, 10,  0, 13]), 预测的目标：61
输入内容：tensor([15, 21, 27, 10,  0, 13, 61]), 预测的目标：39
输入内容：tensor([15, 21, 27, 10,  0, 13, 61, 39]), 预测的目标：63
输入内容：tensor([47]), 预测的目标：50
输入内容：tensor([47, 50]), 预测的目标：50
输入内容：tensor([47, 50, 50]), 预测的目标：5
输入内容：tensor([47, 50, 50,  5]), 预测的目标：58
输入内容：tensor([47, 50, 50,  5, 58]), 预测的目标：1
输入内容：tensor([47, 50, 50,  5, 58,  1]), 预测的目标：54
输入内容：tensor([47, 50, 50,  5, 58,  1, 54]), 预测的目标：50
输入内容：tensor([47, 50, 50,  5, 58,  1, 54, 50]), 预测的目标：

In [73]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
# 2-gram
class BingramLanguageModel(nn.Module):
 
    def __init__(self, vocab_size):
        super().__init__()
        # 每个token都直接从Embedding中查询对应的logits值 以进⾏下⼀个token的推理
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
    
        # idx值和targets值都是整型张量 (B,T)
        logits = self.token_embedding_table(idx) # (B,T,C)
    
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx (B,T) 数组对应着当前的输⼊内容 [1,1]
        for _ in range(max_new_tokens):
            # 模型推理
            logits, loss = self(idx) # (B,T) -> (1,1...100)
            # 获取最后⼀个时间步的输出
            logits = logits[:, -1, :] # (1,100,65) -> (1,65)
            # 应⽤softmax转换为概率值
            probs = F.softmax(logits, dim=-1) # (B,C)
            # 按权重值采样，返回对应的索引
            # idx_next = torch.argmax(probs, dim=-1)
            # 随机采样
            idx_next = torch.multinomial(probs,num_samples=1) # (B,1)
            # 应⽤采样后的索引
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1) [1,2],[1,3]... [1,max_new_tokens]
        return idx

In [74]:
m = BingramLanguageModel(vacob_size)
# 模型训练
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

batch_size = 32
for step in range(1000):

    xb, yb = get_batch('train')

    # 推理计算损失
    logtis, loss = m(xb, yb)
    optimizer.step()
    m.zero_grad(set_to_none=True)

print(loss.item())

# 模型生成

4.821393013000488


In [75]:
# 模型推理

token_idx = torch.zeros((1,1), dtype=torch.long)

result = m.generate(token_idx, 500)
print(decode(result[0].tolist()))


SlKvi-Id?QMckhsc; dcBDWgHLWvvHP zoSBl.RTxIW
r3fqH$srJlTr'qpJjJ!XJASBNpHOYIdQY:livviAcLnpQ!!UzMERFcH?Yh;;Sq.f uslI'TnA
rwgOj$dhPWr,SV?hsujmKpgXXUh;ApmzkHd?hESX&GT;TrJgkiF-oKbXC'A -bnlyNLECHAUQkgCCZM;Tb-gHC-I'td!:wLlHAcPD'bA-.:M.ZtOptXEQcL?hfaoiqbPd?ObPnQQJMs&$aopupIBfGJsZaI'ottlfo..k$W$Akp?yl?ajKlzY!lx&QQLW? tSbXFkyhl-LCVsHUEcOdln$SClgjuk:3Iv
?OqlrV;!PL,fzgyYh
pmRjXSQ&xk!$
o,SiqmZJgKuDny,S$E$fYQGSV-ivvKcOvi-nQGX&q-YQbm dEZJXx;Akr-I&yZ&-wIWcd
RFgXTpDUgM:CK$I!uo'IBT -
j?wfyxqVr.&fiqtRS.ZttxQhOKG'dg


In [14]:
data[:100]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [11]:
# 单词 -》 token index
encode('you')
decode([63, 53, 59])

'you'

In [8]:
itos

{0: '\n',
 1: ' ',
 2: '!',
 3: '$',
 4: '&',
 5: "'",
 6: ',',
 7: '-',
 8: '.',
 9: '3',
 10: ':',
 11: ';',
 12: '?',
 13: 'A',
 14: 'B',
 15: 'C',
 16: 'D',
 17: 'E',
 18: 'F',
 19: 'G',
 20: 'H',
 21: 'I',
 22: 'J',
 23: 'K',
 24: 'L',
 25: 'M',
 26: 'N',
 27: 'O',
 28: 'P',
 29: 'Q',
 30: 'R',
 31: 'S',
 32: 'T',
 33: 'U',
 34: 'V',
 35: 'W',
 36: 'X',
 37: 'Y',
 38: 'Z',
 39: 'a',
 40: 'b',
 41: 'c',
 42: 'd',
 43: 'e',
 44: 'f',
 45: 'g',
 46: 'h',
 47: 'i',
 48: 'j',
 49: 'k',
 50: 'l',
 51: 'm',
 52: 'n',
 53: 'o',
 54: 'p',
 55: 'q',
 56: 'r',
 57: 's',
 58: 't',
 59: 'u',
 60: 'v',
 61: 'w',
 62: 'x',
 63: 'y',
 64: 'z'}

In [63]:
import torch
import torch.nn.functional as F
torch.manual_seed(1771)
wei = torch.randn((4,4))
tril = torch.tril(torch.ones(4,4))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.7685, 0.2315, 0.0000, 0.0000],
        [0.1005, 0.7768, 0.1228, 0.0000],
        [0.1467, 0.5614, 0.2325, 0.0594]])